# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [3]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))

adult_dt.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [7]:
from sklearn.model_selection import train_test_split

#create X and Y df (features, target 'income')
X = adult_dt.drop('income', axis=1)
Y = adult_dt.income

#split data into train and test of 70% and 30%. set random_state to 42
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

#verify the shape of the train and test data
print(f"X_train: {X_train.shape}| X_test: {X_test.shape}| Y_train: {Y_train.shape}| Y_test: {Y_test.shape}")
      

X_train: (22792, 14)| X_test: (9769, 14)| Y_train: (22792,)| Y_test: (9769,)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 

answer: Random state is a parameter of train_test_split. It handles mixing and splitting of data prior to the split.

+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?
It's used like an Identifier to reproduce same results

*(Comment here.)*

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

# 1.) Set numerical and categorical columns
numerical_col = ['age', 'fnlwgt', 'education-num', 
                'capital-gain', 'capital-loss', 'hours-per-week']
categorical_col = ['workclass', 'education', 'marital-status', 
                  'occupation','relationship', 'race', 
                  'sex', 'native-country']

# 2.) Apply KNN-based imputation for numerical columns
impute_knn_numeric = Pipeline([
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

# 3.) Apply mode imputation for categorical columns

impute_simple_categorical = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='infrequent_if_exist', drop='first'))
])

# Create ColumnTransformer
column_transformer = ColumnTransformer(
    transformers=[
        ('num_transforms', impute_knn_numeric, numerical_col),
        ('cat_transforms', impute_simple_categorical, categorical_col)
    ]
)

# Display the ColumnTransformer
print(column_transformer)

ColumnTransformer(transformers=[('num_transforms',
                                 Pipeline(steps=[('imputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('scaler', RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat_transforms',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='infrequent_if_exist'))]),
              

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [10]:
from sklearn.ensemble import RandomForestClassifier

#define Pipeline from ColumnTransformer and RandomForestClassifier
pipe = Pipeline([
    ('preprocess', column_transformer),
    ('classifier', RandomForestClassifier())
])

print(pipe)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [12]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import log_loss, roc_auc_score,accuracy_score, balanced_accuracy_score

#report the training and validation performance using cross_validate
scoring = ['accuracy', 'balanced_accuracy', 'roc_auc', 'neg_log_loss']
cv_results = cross_validate(pipe, X_train, Y_train, cv=5, scoring=scoring,return_train_score=True)

print(cv_results)


c:\Users\zai_m\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\zai_m\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


{'fit_time': array([7.88854194, 7.96351361, 8.15544772, 8.30582619, 8.19565773]), 'score_time': array([0.2111752 , 0.17823744, 0.21239042, 0.21014047, 0.21354795]), 'test_accuracy': array([0.85062514, 0.8438254 , 0.85103115, 0.8571742 , 0.85870996]), 'train_accuracy': array([1.        , 0.99994515, 0.99983547, 1.        , 0.99994516]), 'test_balanced_accuracy': array([0.77386975, 0.76169895, 0.77122038, 0.77865939, 0.78028837]), 'train_balanced_accuracy': array([1.        , 0.9998869 , 0.99981453, 1.        , 0.99988693]), 'test_roc_auc': array([0.90601471, 0.90253293, 0.90366158, 0.90609202, 0.90151432]), 'train_roc_auc': array([1.        , 1.        , 0.99999997, 1.        , 1.        ]), 'test_neg_log_loss': array([-0.35447044, -0.37336184, -0.3583239 , -0.38516871, -0.38148056]), 'train_neg_log_loss': array([-0.08142653, -0.08123631, -0.08187186, -0.08214096, -0.08146282])}


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [13]:
cv_results_df = pd.DataFrame(cv_results).sort_values(by='test_neg_log_loss', ascending=False)

print(cv_results_df)

   fit_time  score_time  test_accuracy  train_accuracy  \
0  7.888542    0.211175       0.850625        1.000000   
2  8.155448    0.212390       0.851031        0.999835   
1  7.963514    0.178237       0.843825        0.999945   
4  8.195658    0.213548       0.858710        0.999945   
3  8.305826    0.210140       0.857174        1.000000   

   test_balanced_accuracy  train_balanced_accuracy  test_roc_auc  \
0                0.773870                 1.000000      0.906015   
2                0.771220                 0.999815      0.903662   
1                0.761699                 0.999887      0.902533   
4                0.780288                 0.999887      0.901514   
3                0.778659                 1.000000      0.906092   

   train_roc_auc  test_neg_log_loss  train_neg_log_loss  
0            1.0          -0.354470           -0.081427  
2            1.0          -0.358324           -0.081872  
1            1.0          -0.373362           -0.081236  
4         

Calculate the mean of each metric. 

In [14]:
cv_results_df.mean()

fit_time                   8.101797
score_time                 0.205098
test_accuracy              0.852273
train_accuracy             0.999945
test_balanced_accuracy     0.773147
train_balanced_accuracy    0.999918
test_roc_auc               0.903963
train_roc_auc              1.000000
test_neg_log_loss         -0.370561
train_neg_log_loss        -0.081628
dtype: float64

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [16]:
pipe.fit(X_train, Y_train)

Y_pred_train = pipe.predict(X_train)
Y_pred_test = pipe.predict(X_test)
Y_proba_train = pipe.predict_proba(X_train)
Y_proba_test = pipe.predict_proba(X_test)

print(Y_pred_train,Y_proba_test)

cv_results = {
    'log_loss_train': log_loss(Y_train, Y_proba_train) * -1,
    'log_loss_test': log_loss(Y_test, Y_proba_test) * -1,
    'roc_auc_train': roc_auc_score(Y_train, Y_proba_train[:,1]),
    'roc_auc_test': roc_auc_score(Y_test, Y_proba_test[:,1]),
    'accuracy_train': accuracy_score(Y_train, Y_pred_train),
    'accuracy_test': accuracy_score(Y_test, Y_pred_test),
    'balanced_accuracy_train': balanced_accuracy_score(Y_train, Y_pred_train),
    'balanced_accuracy_test': balanced_accuracy_score(Y_test, Y_pred_test)
}

print(cv_results)

[0 1 0 ... 0 0 0] [[0.99 0.01]
 [0.52 0.48]
 [0.25 0.75]
 ...
 [0.99 0.01]
 [0.4  0.6 ]
 [0.   1.  ]]
{'log_loss_train': -0.08039515960016323, 'log_loss_test': -0.38406343043649227, 'roc_auc_train': 1.0, 'roc_auc_test': 0.9012647478069222, 'accuracy_train': 1.0, 'accuracy_test': 0.8537209540382844, 'balanced_accuracy_train': 1.0, 'balanced_accuracy_test': 0.7724361148162382}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

answer: It's convenient because it's required for computing other metrics

(Answer here.)

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.